In [37]:
import sympy as sp
import numpy as np
import itertools
from sympy import satisfiable, And, Eq, Le, Ge, Add, Expr
from IPython.display import display
from scipy.optimize import linprog
from typing import List

In [38]:
def get_all_free_symbols(expr_list: List[Expr], region_list: List[List[Expr]] = None):
    """
    Given a list of SymPy expressions, return a list of all unique free symbols.
    """
    all_symbols = set()
    for expr in expr_list:
        all_symbols.update(expr.free_symbols)
        
    if region_list:
        for region in region_list:
            for expr in region:
                all_symbols.update(expr.free_symbols)
    return list(all_symbols)

In [39]:
def affine_expr(coeffs, symbols):
    return sum(c * s for c, s in zip(coeffs[:-1], symbols)) + coeffs[-1]

In [40]:
def compute_all_sf_expressions_with_regions(
    t1_bounds_array, t2_bounds_array,
    theta1_critical_regions, theta2_critical_regions,
    joint_pdf_expr, d_syms, n_gl, xi_1, wi_1, xi_2, wi_2,
    theta_1, theta_2
):
    n_t1_regions = t1_bounds_array.shape[0]
    n_t2_regions = t2_bounds_array.shape[0]
    n_q1, n_q2 = n_gl

    t2_region_choices = list(itertools.product(range(n_t2_regions), repeat=n_q1))

    sf_exprs = []
    sf_regions = []

    for t1_region_idx in range(n_t1_regions):
        t1_min_expr = affine_expr(t1_bounds_array[t1_region_idx, 0], d_syms)
        t1_max_expr = affine_expr(t1_bounds_array[t1_region_idx, 1], d_syms)

        theta1_points = []
        for i in range(n_q1):
            t1 = 0.5 * (t1_max_expr - t1_min_expr) * xi_1[i] + 0.5 * (t1_max_expr + t1_min_expr)
            theta1_points.append(sp.simplify(t1))

        # Get constraints for this θ₁ region
        t1_region_constraints = theta1_critical_regions[t1_region_idx]
        t1_ineqs = []
        for row in t1_region_constraints:
            ineq = sum(c * d_i for c, d_i in zip(row[:-1], d_syms)) + row[-1] <= 0
            t1_ineqs.append(ineq)

        for t2_combo in t2_region_choices:
            sf_sum = 0
            region_ineqs = list(t1_ineqs)

            for i in range(n_q1):
                t1 = theta1_points[i]
                t2_region_idx = t2_combo[i]

                t2_min_expr = affine_expr(t2_bounds_array[t2_region_idx, 0], [t1] + list(d_syms))
                t2_max_expr = affine_expr(t2_bounds_array[t2_region_idx, 1], [t1] + list(d_syms))

                # Get θ₂ constraints (θ₁ + d terms)
                t2_constraints = theta2_critical_regions[t2_region_idx]
                for row in t2_constraints:
                    theta1_coeff = row[0]
                    d_coeffs = row[1:-1]
                    const = row[-1]
                    ineq = theta1_coeff * t1 + sum(c * d_i for c, d_i in zip(d_coeffs, d_syms)) + const <= 0
                    region_ineqs.append(sp.simplify(ineq))

                for j in range(n_q2):
                    t2 = 0.5 * (t2_max_expr - t2_min_expr) * xi_2[j] + 0.5 * (t2_max_expr + t2_min_expr)
                    pdf_val = joint_pdf_expr.subs({theta_1: t1, theta_2: sp.simplify(t2)})
                    weight = wi_1[i] * wi_2[j]
                    scale = 0.25 * (t1_max_expr - t1_min_expr) * (t2_max_expr - t2_min_expr)
                    sf_sum += weight * scale * pdf_val

            sf_exprs.append(sp.simplify(sf_sum))
            sf_regions.append(region_ineqs)

    return sf_exprs, sf_regions

In [41]:
def extract_linprog_form(ineqs, variables):
    A_ub = []
    b_ub = []

    for ineq in ineqs:
        # Normalize: expr <= 0 form
        if isinstance(ineq, Le):  # lhs <= rhs → lhs - rhs <= 0
            expr = ineq.lhs - ineq.rhs
        elif isinstance(ineq, Ge):  # lhs >= rhs → rhs - lhs <= 0
            expr = ineq.rhs - ineq.lhs
        else:
            raise ValueError(f"Unsupported type: {ineq}")

        coeffs = [expr.coeff(v) for v in variables]
        const = -expr.subs({v: 0 for v in variables})
        A_ub.append([float(c) for c in coeffs])
        b_ub.append(float(const))

    return np.array(A_ub), np.array(b_ub)

In [42]:
# Design variables
m = 2
d_syms = sp.symbols(f'd0:{m}')  # d0, d1
theta_1, theta_2 = sp.symbols('theta_1 theta_2')

# Gauss–Legendre quadrature settings
n_gl = [1, 1]
xi_1, wi_1 = np.polynomial.legendre.leggauss(n_gl[0])
xi_2, wi_2 = np.polynomial.legendre.leggauss(n_gl[1])

# Bounds arrays
theta1_bounds_array = np.array([
    [[0.0, 0.0, 0.0], [0.0, 0.0, 4.0]],  # d0 ± 10
    [[0.75, -1.5, -1.25], [0.0, 0.0, 4.0]]  # d1 ± 5
])

theta2_bounds_array = np.array([
    [[-8/3, 2.0, -4.0, 2/3], [0.0, 0.0, 0.0, 4.0]],  # θ1 ± d2
    [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0]],  # θ1 ± 2d2
    [[1/3, -0.5, 0.5, -1/3], [0.0, 0.0, 0.0, 4.0]]  # θ1 ± 3d2
])

# PDF
joint_pdf_expr = (2 / sp.pi) * sp.exp(-2 * ((theta_1 - 2) ** 2 + (theta_2 - 2) ** 2))

# Critical regions (CRs)
# theta1_critical_regions: constraints in terms of d (shape: n_regions × n_ineqs × (m+1))
theta1_critical_regions = np.array([
    [[1, -2, -5/3]],
    [[-1, 2, -5/3]]
], dtype=object)

# theta2_critical_regions: constraints in terms of [theta1, d0, d1, const]
theta2_critical_regions = np.array([
    [[-8/3, 2, -4, -10/3], [8/3, -2, 4, -2/3]],
    [[8/3, -4.0, 4.0, -8/3], [-8/3, 2, -4, 2/3]],
    [[-8/3, 4, -4, 8/3]]
], dtype=object)


In [43]:
sf_exprs, sf_regions = compute_all_sf_expressions_with_regions(
    theta1_bounds_array, theta2_bounds_array,
    theta1_critical_regions, theta2_critical_regions,
    joint_pdf_expr, d_syms, n_gl,
    xi_1, wi_1, xi_2, wi_2,
    theta_1, theta_2
)

In [44]:
print(f'Number of SF expressions: {len(sf_exprs)}')
print(f'Number of critical regions: {len(sf_regions)}')

Number of SF expressions: 6
Number of critical regions: 6


In [61]:
for i, ex in enumerate(sf_exprs):
    print(f'SF expression {i}')
    display(ex)
    print('\n')

SF expression 0


(-16.0*d0 + 32.0*d1 + 69.3333333333333)*exp(-2*(-1.0*d0 + 2.0*d1 + 2.33333333333333)**2)/pi



SF expression 1


32.0/pi



SF expression 2


...

8.0*(-1.0*d0 + 2.0*d1 + 7.0)*(-0.1875*d0 + 0.375*d1 + 1.3125)*exp(-2*(0.375*d0 - 0.75*d1 - 0.625)**2 - 2*(0.5*d0 - 1.0*d1 - 1.5)**2)/pi



SF expression 4


(-6.0*d0 + 12.0*d1 + 42.0)*exp(-2*(-0.375*d0 + 0.75*d1 + 0.625)**2)/pi



SF expression 5


8.0*(-0.1875*d0 + 0.375*d1 + 1.3125)*(0.375*d0 - 0.25*d1 + 3.875)*exp(-2*(-0.1875*d0 + 0.125*d1 + 0.0625)**2 - 2*(0.375*d0 - 0.75*d1 - 0.625)**2)/pi

In [59]:
for i, region_exprs in enumerate(sf_regions):
    print(f'For region {i}:')
    for expr in region_exprs:
        display(expr)
    print('\n')
# sf_regions

For region 0:


d0 - 2*d1 - 1.66666666666667 <= 0

d0 - 2*d1 <= 4.33333333333333

d0 - 2*d1 >= 2.33333333333333



For region 1:


...

3.0*d0 - 2.0*d1 >= 1.0

1.0*d0 - 2.0*d1 <= 3.0



For region 5:


-d0 + 2*d1 - 1.66666666666667 <= 0

3.0*d0 - 2.0*d1 <= 1.0

In [47]:
d0, d1 = sp.symbols('d0 d1')
vars = [d0, d1]

In [48]:
feasibility = list()
for exprs in sf_regions:
    A_ub, b_ub = extract_linprog_form(exprs, vars)
    res = linprog(c=[0]*len(vars), A_ub=A_ub, b_ub=b_ub)
    # print(res.success)
    feasibility.append(res.success)

In [49]:
feasibility

[False, True, True, True, True, True]